# 사용자 맞춤 뉴스 추천 모델링

In [16]:
import pandas as pd
import numpy as np

In [18]:
df_bosa = pd.read_csv("./bosa_news_all.csv")
df_bosa.head()

FileNotFoundError: [Errno 2] No such file or directory: './bosa_news_all.csv'

In [3]:
df_bosa.drop(columns=['_id'], inplace=True)
df_bosa.dropna(subset=['news_title'], inplace=True)
df_bosa.head()

,news_title,news_when,news_topic,news_contents,news_url
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",02.29 11:57,다국적제약/의료기기,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...,http://www.bosa.co.kr/news/articleView.html?id...
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",02.29 10:52,제약산업,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...,http://www.bosa.co.kr/news/articleView.html?id...
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",02.29 10:29,동정,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...,http://www.bosa.co.kr/news/articleView.html?id...
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",02.29 10:07,제약산업,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을...",http://www.bosa.co.kr/news/articleView.html?id...
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,02.29 05:50,학회/학술,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...,http://www.bosa.co.kr/news/articleView.html?id...


In [4]:
df_bosa['news_topic'].value_counts()  #제약/개발 #병원 #법령/정책 #의료기기 #학회/학술 #

news_topic
제약산업          449
제약            331
기타기관          233
병원            199
병의원           194
기획연재           87
복지부            87
국회             60
다국적제약/의료기기     58
학회/학술          58
제약·유통          56
개원가            51
식약처            47
바이오            46
동정             45
의료             40
의료기기·IT        38
질병청            36
심평원            31
건보공단           19
의약정책           16
의료단체           15
약사단체           11
유통              9
식품              8
협회              6
진흥원             4
인사              4
의료기기            3
책소개             3
단신              3
정책·행정           3
약사·약국           2
포토뉴스            2
너몰내알            1
보사연             1
해외              1
의원·병원           1
치과/한의사          1
화장품             1
약대/학술           1
Name: count, dtype: int64

In [5]:
df_bosa[df_bosa.isnull().any(axis=1)]

,news_title,news_when,news_topic,news_contents,news_url
1472,"아주대의료원, 국립의료원과 상호지원 및 협력 협약 체결",2016.07.27 10:05,병의원,NaN,http://www.bosa.co.kr/news/articleView.html?id...
1476,"이지원 교수, 신장학회서 우수연구자상 수상",2016.07.14 11:14,동정,NaN,http://www.bosa.co.kr/news/articleView.html?id...
1491,마크로젠과 유전체 정밀의학 공동연구 MOU,2016.04.28 10:53,병의원,NaN,http://www.bosa.co.kr/news/articleView.html?id...
1501,2016 제1차 여성건강포럼 개최,2016.03.10 10:02,동정,NaN,http://www.bosa.co.kr/news/articleView.html?id...
1505,"순천향대 부천병원, 극희귀질환 진단기관 선정",2016.03.02 09:38,병의원,NaN,http://www.bosa.co.kr/news/articleView.html?id...
...,...,...,...,...,...
2256,먹지도 굶지도 못하는 희귀질환 발견,2001.04.11 15:07,학회/학술,NaN,http://www.bosa.co.kr/news/articleView.html?id...
2257,희귀의약품센터 공익성 기부금 대상단체 지정,2001.01.16 12:38,개원가,NaN,http://www.bosa.co.kr/news/articleView.html?id...
2258,쇼그렌증후군 관여 단백질 발견,2001.05.24 18:32,의료,NaN,http://www.bosa.co.kr/news/articleView.html?id...
2259,26일 `희귀질환 치료' 심포지엄,2001.05.22 19:35,학회/학술,NaN,http://www.bosa.co.kr/news/articleView.html?id...


In [6]:
df_bosa.dropna(subset=['news_contents'], inplace=True)

In [7]:
df_bosa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2142 entries, 0 to 2229
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_title     2142 non-null   object
 1   news_when      2142 non-null   object
 2   news_topic     2142 non-null   object
 3   news_contents  2142 non-null   object
 4   news_url       2142 non-null   object
dtypes: object(5)
memory usage: 100.4+ KB


In [8]:
df_bosa_test1 = df_bosa.copy()

### 토큰화

In [390]:
# 불용어 리스트 생성
stopwords = ['치료', '되다', '환자', '개발', '의료', '이지은', '이프', '고어', '햇반'] #추가 생성 필요
f=open('./csv/korean_stopwords_basic.txt') #기본적으로 제공되는 한국어 불용어 리스트 파일
lines = f.readlines()
for line in lines:
    line = line.strip()
    stopwords.append(line)
f.close()

In [409]:
f2=open('./csv/bosa_news_stopword.txt')
lines = f2.readlines()
for line in lines:
    line = line.strip()
    stopwords.append(line)
f2.close()

In [410]:
from konlpy.tag import Okt
okt = Okt()
#토크나이징 함수 정의
def tokenizer(raw, pos=['Noun', 'Verb'], stopword=stopwords):
    return [
        word for word, tag in okt.pos(raw, norm=True, stem=True)
        if len(word) >1 and tag in pos and word not in stopword
    ]

## 준지도 학습

### 1. 레이블이 있는 데이터로 모델 학습(분류모델)

In [411]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=3)

In [412]:
features = tfidfvectorizer.fit_transform(df_bosa['news_contents'])
# news_title에 있는 내용을 백터&토큰화

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [413]:
features

<2142x7330 sparse matrix of type '<class 'numpy.float64'>'
	with 263780 stored elements in Compressed Sparse Row format>

### 군집 만들기

In [414]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.decomposition import LatentDirichletAllocation

In [415]:
components= LatentDirichletAllocation(n_components=4, n_jobs=-1)
components.fit(features)

LatentDirichletAllocation(n_components=4, n_jobs=-1)

In [416]:
dictionary_list = tfidfvectorizer.get_feature_names_out(features)
dictionary_list

array(['가가', '가게', '가격', ..., '힐링', '힐튼', '힘쓰다'], dtype=object)

In [417]:
components.components_ # 토픽별로 단어의 확률 분포를 나타냄

array([[2.34905078, 0.25011132, 8.73064861, ..., 0.25011806, 0.25326132,
        0.25147989],
       [1.54403296, 0.70319979, 4.26926166, ..., 1.40449277, 0.78277368,
        3.34978647],
       [0.25045245, 0.25036854, 0.25038843, ..., 0.25027809, 0.25036896,
        0.25024741],
       [0.25045237, 0.25036848, 0.25038836, ..., 0.25027804, 0.25036889,
        0.25024736]])

In [418]:
topics_output = components.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [419]:
df_topics_score

,0,1,2,3
0,0.029738,0.912989,0.028637,0.028637
1,0.029604,0.918148,0.026124,0.026124
2,0.030319,0.909815,0.029933,0.029933
3,0.030113,0.911362,0.029262,0.029262
4,0.020330,0.940118,0.019776,0.019776
...,...,...,...,...
2137,0.035304,0.897201,0.033748,0.033748
2138,0.024355,0.927561,0.024042,0.024042
2139,0.035873,0.894426,0.034850,0.034850
2140,0.050031,0.850768,0.049600,0.049600


In [420]:
import numpy as np
df_topics_score['topic_num'] = np.argmax(topics_output, axis=1)

In [421]:
df_topics_score['news_title'] = df_bosa['news_title']
df_topics_score

,0,1,2,3,topic_num,news_title
0,0.029738,0.912989,0.028637,0.028637,1,"한국쿄와기린, 세계 희귀질환의 날 행사 참여"
1,0.029604,0.918148,0.026124,0.026124,1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최"
2,0.030319,0.909815,0.029933,0.029933,1,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상"
3,0.030113,0.911362,0.029262,0.029262,1,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시"
4,0.020330,0.940118,0.019776,0.019776,1,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다
...,...,...,...,...,...,...
2137,0.035304,0.897201,0.033748,0.033748,1,병원산책| 영동세브란스병원 '호흡재활의 힘' 발휘
2138,0.024355,0.927561,0.024042,0.024042,1,27일 '수포성표피박리증' 집중조명
2139,0.035873,0.894426,0.034850,0.034850,1,질병관리본부 '희귀·난치성질환' 이미지 제고
2140,0.050031,0.850768,0.049600,0.049600,1,의료기관 종별 구분 3단계 추진 탄력받나


In [422]:
# 그럼 이제 각각의 news가 어떤 topic으로 분류되는지는 알았는데
# 각 토픽이 어떤 단어를 가지고 있는지 알기 위해서는 상위 단어 추출이 필요함

# 상위 단어 추출

topics_list = []
for topic in components.components_ : # 토픽별로 단어의 확률 분포 나타낸 것
    df_datas = [topic, dictionary_list] # dictionary_list는 단어 나열
    df_topics = pd.DataFrame(data=df_datas).T
    df_topics = df_topics.sort_values(0, ascending=False)
    topics_text = ' '.join(df_topics[1].values[:5]) # 상위 4개를 시리즈 형식으로 출력하기
    print(topics_text) 
    topics_list.append(topics_text)

topics_list_table = [topics_list, ['Topic1', 'Topic2', 'Topic3', 'Topic4']]
topics_list_table

의약품 바이오 신약 미국 승인
질환 병원 연구 진단 교수
의약외품 수재 시장점유율 서식 성법
영진약품 의약외품 수재 시장점유율 서식


[['의약품 바이오 신약 미국 승인',
  '질환 병원 연구 진단 교수',
  '의약외품 수재 시장점유율 서식 성법',
  '영진약품 의약외품 수재 시장점유율 서식'],
 ['Topic1', 'Topic2', 'Topic3', 'Topic4']]

In [423]:
topics_list_table = pd.DataFrame(topics_list_table)
topics_list_table

,0,1,2,3
0,의약품 바이오 신약 미국 승인,질환 병원 연구 진단 교수,의약외품 수재 시장점유율 서식 성법,영진약품 의약외품 수재 시장점유율 서식
1,Topic1,Topic2,Topic3,Topic4


In [424]:
topics_list_table[0][0]

'의약품 바이오 신약 미국 승인'

In [425]:
pivot_table_contents = pd.pivot_table(data=df_topics_score,
                                   values='news_title'
                                   ,index='topic_num'
                                   , aggfunc='count')
pivot_table_contents

,news_title
topic_num,
0,493
1,1593


### 시각화

In [426]:
vis = pyLDAvis.lda_model.prepare(components, features, tfidfvectorizer)
# 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [427]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소

In [428]:
df_bosa_test1['topic_model'] = df_topics_score['topic_num']
df_bosa_test1.head()

,news_title,news_when,news_topic,news_contents,news_url,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",02.29 11:57,다국적제약/의료기기,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...,http://www.bosa.co.kr/news/articleView.html?id...,1.0
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",02.29 10:52,제약산업,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...,http://www.bosa.co.kr/news/articleView.html?id...,1.0
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",02.29 10:29,동정,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...,http://www.bosa.co.kr/news/articleView.html?id...,1.0
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",02.29 10:07,제약산업,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을...",http://www.bosa.co.kr/news/articleView.html?id...,1.0
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,02.29 05:50,학회/학술,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...,http://www.bosa.co.kr/news/articleView.html?id...,1.0


In [429]:
type(df_bosa_test1['topic_model'][0])

numpy.float64

In [430]:
for idx, topic in enumerate(df_bosa_test1['topic_model']) :
    if topic == 0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[0][0]
    elif topic == 1.0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[1][0]
    elif topic == 2.0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[2][0]
    elif topic == 3.0 :
        df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[3][0]
df_bosa_test1.head()

/tmp/ipykernel_12292/1707612536.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '질환 병원 연구 진단 교수' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table[1][0]


,news_title,news_when,news_topic,news_contents,news_url,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",02.29 11:57,다국적제약/의료기기,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...,http://www.bosa.co.kr/news/articleView.html?id...,질환 병원 연구 진단 교수
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",02.29 10:52,제약산업,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...,http://www.bosa.co.kr/news/articleView.html?id...,질환 병원 연구 진단 교수
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",02.29 10:29,동정,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...,http://www.bosa.co.kr/news/articleView.html?id...,질환 병원 연구 진단 교수
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",02.29 10:07,제약산업,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을...",http://www.bosa.co.kr/news/articleView.html?id...,질환 병원 연구 진단 교수
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,02.29 05:50,학회/학술,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...,http://www.bosa.co.kr/news/articleView.html?id...,질환 병원 연구 진단 교수


## 시도2 : 전처리 후 다시 군집화하기

### news_topic 전처리

In [26]:
df_bosa.head()

,news_title,news_url,news_when,news_topic
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,02.29 11:57,다국적제약/의료기기
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:52,제약산업
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:29,동정
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,02.29 10:07,제약산업
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,02.29 05:50,학회/학술


In [27]:
df_bosa['news_topic'].value_counts()

news_topic
제약산업          472
제약            360
기타기관          251
병원            212
병의원           206
복지부            95
기획연재           91
제약·유통          67
다국적제약/의료기기     62
국회             61
학회/학술          60
개원가            57
식약처            54
바이오            52
동정             47
의료             41
의료기기·IT        40
질병청            37
심평원            33
의약정책           19
건보공단           19
의료단체           16
약사단체           12
유통             10
식품              8
협회              8
진흥원             5
책소개             5
포토뉴스            4
인사              4
단신              3
정책·행정           3
의료기기            3
약사·약국           2
너몰내알            1
보사연             1
치과/한의사          1
의원·병원           1
화장품             1
약대/학술           1
Name: count, dtype: int64

In [29]:
df_bosa.loc[df_bosa['news_topic'] == '기타기관']

,news_title,news_url,news_when,news_topic
113,"NECA, 제2차 '제한적 의료기술' 신청 접수 재공고",http://www.bosa.co.kr/news/articleView.html?id...,2023.11.10 09:56,기타기관
174,"보의연, 2023년 제2차 ‘제한적 의료기술’ 신청 접수",http://www.bosa.co.kr/news/articleView.html?id...,2023.09.27 17:02,기타기관
262,"NECA, ‘2023년 1차 제한적 의료기술’ 신청접수",http://www.bosa.co.kr/news/articleView.html?id...,2023.06.05 10:03,기타기관
344,보건의료 200여개 법령 ‘읽기 쉬운 문장’ 개선 추진,http://www.bosa.co.kr/news/articleView.html?id...,2023.03.17 06:00,기타기관
348,"케이메디, KAIST 희귀질환 파브리병 치료제 개발 연구 지원",http://www.bosa.co.kr/news/articleView.html?id...,2023.03.14 14:53,기타기관
...,...,...,...,...
2406,만성·희귀질환자 등 의료급여 확대,http://www.bosa.co.kr/news/articleView.html?id...,2002.05.20 10:38,기타기관
2410,고혈압 등 10개질환 급여일수 30일 연장,http://www.bosa.co.kr/news/articleView.html?id...,2001.12.12 11:33,기타기관
2413,혈소판감소성 자반증치료제 희귀약 지정,http://www.bosa.co.kr/news/articleView.html?id...,2001.11.06 10:50,기타기관
2415,“희귀의약품센터는 예산 은닉처(?)”,http://www.bosa.co.kr/news/articleView.html?id...,2001.09.11 12:04,기타기관


In [63]:
df_bosa.all

<bound method DataFrame.all of                                  news_title  \
0                  한국쿄와기린, 세계 희귀질환의 날 행사 참여   
1        광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최   
2          순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상   
3          창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시   
4     [세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다   
...                                     ...   
2421                     26일 `희귀질환 치료' 심포지엄   
2422                  `희귀^난치성질환자 연합회' 구성 추진   
2423                      책임운영기관 평가 “적정성 결여   
2424                    먹지도 굶지도 못하는 희귀질환 발견   
2425                희귀의약품센터 공익성 기부금 대상단체 지정   

                                               news_url         news_when  \
0     http://www.bosa.co.kr/news/articleView.html?id...       02.29 11:57   
1     http://www.bosa.co.kr/news/articleView.html?id...       02.29 10:52   
2     http://www.bosa.co.kr/news/articleView.html?id...       02.29 10:29   
3     http://www.bosa.co.kr/news/articleView.html?id...       02.29 10:07   
4     http://www.bo